# Работа с clickhouse-driver

In [1]:
!pip install clickhouse-driver

     |████████████████████████████████| 501kB 410kB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/e7/f9/f0b53f88060247251bf481fa6ea62cd0d25bf1b11a87888e53ce5b7c8ad2/pytz-2019.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ef/99/53bd1ac9349262f59c1c421d8fcc2559ae8a5eeffed9202684756b648d33/tzlocal-2.0.0-py2.py3-none-any.whl


In [26]:
from clickhouse_driver import Client
client = Client(host='localhost', port=9990)

In [27]:
client.execute('SHOW DATABASES')

[('default',), ('system',)]

In [5]:
client.execute('SELECT * FROM system.numbers LIMIT 5')

[(0,), (1,), (2,), (3,), (4,)]

In [6]:
from datetime import date
client.execute(
    'SELECT %(date)s, %(a)s + %(b)s',
    {'date': date.today(), 'a': 1, 'b': 2})

[('2019-11-03', 3)]

In [9]:
client.execute(
    'CREATE DATABASE IF NOT EXISTS hr'
)

[]

In [13]:
client.execute(
    '''
    CREATE TABLE IF NOT EXISTS hr.events (
        event_date  Date,
        event_time  DateTime,
        user_uid    String
    ) engine=MergeTree(event_date, (user_uid), 8192)'''
)

[]

In [24]:
client.execute(
    'SELECT * FROM hr.events;'
)

[(datetime.date(2019, 11, 3),
  datetime.datetime(2019, 11, 3, 11, 11, 28),
  'test_user'),
 (datetime.date(2019, 11, 3),
  datetime.datetime(2019, 11, 3, 11, 11, 15),
  'test_user'),
 (datetime.date(2019, 11, 3),
  datetime.datetime(2019, 11, 3, 11, 11, 29),
  'test_user'),
 (datetime.date(2019, 11, 3),
  datetime.datetime(2019, 11, 3, 11, 11, 27),
  'test_user')]

In [23]:
from datetime import date, datetime
client.execute(
    'INSERT INTO hr.events (event_date, event_time, user_uid) VALUES',
    [{
        "event_date": datetime.now(),
        "event_time": datetime.now(),
        "user_uid": "test_user",
    }]
)

1

In [25]:
client.execute(
    'SELECT count(*) FROM hr.events;'
)

[(4,)]